載入數據集資料

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
'''
dataset_dict = load_dataset("csv",data_files="./ChnSentiCorp_htl_all.csv") #split不指定會傳出DatasetDict
dataset_dict['train']
'''
#整合上面2行成為1行的語法
dataset = load_dataset("csv",data_files='./ChnSentiCorp_htl_all.csv', split="train")
dataset
#清理資料
dataset = dataset.filter(lambda example: example['review'] is not None)
dataset

Filter:   0%|          | 0/7766 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

數據集分類

In [10]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

數據集分詞處理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

def tokenizer_process(example:dict[str,any]) -> dict[str,any]:
    tokenized_example:dict = tokenizer(example['review'], max_length=128, truncation=True)
    tokenized_example['labels'] = example['label']
    return tokenized_example

tokenized_datasets = datasets.map(tokenizer_process,batched=True,remove_columns=datasets['train'].column_names)
tokenized_datasets

取得預訓練模型

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-chinese')

建立評估函數

In [13]:
import evaluate
#acc_metric = evaluate.load('accuracy')
#f1_metric = evaluate.load('f1')
acc_metric = evaluate.load('evaluate-main/metrics/accuracy/accuracy.py')
f1_metric = evaluate.load('evaluate-main/metrics/f1/f1.py')

def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc
